# Absher Hackathon

### YAQITH Sentinel – Behavioral AI Security Agent Project

#### Team Name: SANAM
#### Team Members:
1. Majd Alqarni
2. Eman Aldosari
3. Arwa Alkibari
4. Alwalah Awaji

***





#### YAQITH Sentinel – Behavioral AI Security Agent

#### YAQITH Sentinel is an AI-powered security agent designed to enhance digital identity protection inside the Absher platform by combining official digital identity with behavioral biometrics.

#### The system analyzes multiple real-time signals—such as device identity, IP address, location, typing patterns, navigation timing, and login history—to verify whether the user is the legitimate account owner.

#### What Problem Does It Solve?

#### Traditional authentication (password, OTP, biometrics) can still be bypassed through identity theft or device spoofing.

#### YAQITH adds a second behavioral layer that is extremely difficult to fake, providing continuous and intelligent authentication.

#### How It Works

#### The agent performs three core functions:

#### Data Collection Device type & browser
#### IP address & location

#### Login timestamps

#### Behavioral signals (typing speed, navigation rhythm, touch patterns)

#### Anomaly Detection
#### A lightweight ML model analyzes the user’s normal behavior and detects unusual activity.

#### Risk Scoring & Action
#### Based on the risk score:

####    1. Low risk: seamless login

####    2. Medium risk: request additional verification

####    3. High risk: block or escalate the session

#### Key Features Real-time behavioral authentication

####    1. Detects suspicious logins with high accuracy

####    2. Privacy-preserving (anonymized behavior patterns)

####    3. Adds security without affecting user experience

### 1. Importing Packages & Columns Identification

In [ ]:
# Importing all necessary packages for ML Model

import os
import json
import joblib
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
# Loading the Dataset
DATA_PATH = "yaqith_sentinel_synthetic_logins_All.xlsx"


In [ ]:
# Model Folder & File
MODEL_DIR = "/content/models"
MODEL_PATH = os.path.join(MODEL_DIR, "risk_model_pipeline.joblib")
FEATURES_PATH = os.path.join(MODEL_DIR, "yaqith_features.json")

In [ ]:
# Numeric Columns
NUMERIC_FEATURES = [
    "is_vpn",
    "is_new_device",
    "is_new_city",
    "distance_from_last_login_km",
    "hour_of_day",
    "weekday",
    "failed_logins_last_24h",
    "typing_speed_cps",
    "nav_pattern_similarity",
    "session_duration_sec",
    "past_compromised_account",
]

# Categorical Columns
CATEGORICAL_FEATURES = [
    "device_type",
    "city",
    "country",
]

# Target Column
TARGET_COL = "is_attack_label"

### 2. Dataset Loading

In [ ]:
def load_data(path: str) -> pd.DataFrame:
    if not os.path.exists(path):
        raise FileNotFoundError(f"لم يتم العثور على الملف: {path}")
    df = pd.read_excel(path)
    return df


def preprocess(df: pd.DataFrame) -> pd.DataFrame:

    required_cols = NUMERIC_FEATURES + CATEGORICAL_FEATURES + [TARGET_COL]
    missing = [col for col in required_cols if col not in df.columns]
    if missing:
        raise ValueError(f"هذه الأعمدة ناقصة في البيانات: {missing}")

    df_proc = df.copy()

    # Convert numeric columns to numbers
    for col in NUMERIC_FEATURES + [TARGET_COL]:
        df_proc[col] = pd.to_numeric(df_proc[col], errors="coerce")

    # Drop All the rows that have (NA)
    df_proc = df_proc.dropna(subset=required_cols)

    # Ensure that the (TARGET_COL) in interger format
    df_proc[TARGET_COL] = df_proc[TARGET_COL].astype(int)

    return df_proc


### 3. Building Model Pipeline

In [ ]:
def build_model_pipeline() -> Pipeline:

    # numeric = passthrough, categorical = OneHot
    preprocessor = ColumnTransformer(
        transformers=[
            ("num", "passthrough", NUMERIC_FEATURES),
            ("cat", OneHotEncoder(handle_unknown="ignore"), CATEGORICAL_FEATURES),
        ]
    )

    rf = RandomForestClassifier(
        n_estimators=200,
        max_depth=None,
        random_state=42,
        class_weight="balanced",
        n_jobs=-1,
    )

    pipeline = Pipeline(
        steps=[
            ("preprocessor", preprocessor),
            ("model", rf),
        ]
    )

    return pipeline


### 4. Model Training & Performance Evaluation

In [ ]:
def train_and_evaluate(df: pd.DataFrame) -> Pipeline:

    X = df[NUMERIC_FEATURES + CATEGORICAL_FEATURES]
    y = df[TARGET_COL]

    X_train, X_test, y_train, y_test = train_test_split(
        X,
        y,
        test_size=0.25,
        random_state=42,
        stratify=y,
    )

    pipeline = build_model_pipeline()

    print("🚀 Training model...")
    pipeline.fit(X_train, y_train)

    y_pred = pipeline.predict(X_test)

    print("\n=== Confusion Matrix ===")
    print(confusion_matrix(y_test, y_pred))

    print("\n=== Classification Report ===")
    print(classification_report(y_test, y_pred, digits=3))

    return pipeline


### 5. Store The Pipeline & Features Info

In [ ]:
def save_artifacts(pipeline: Pipeline) -> None:

    os.makedirs(MODEL_DIR, exist_ok=True)

    joblib.dump(pipeline, MODEL_PATH)
    print(f"\n✅ تم حفظ الـ pipeline في: {MODEL_PATH}")

    with open(FEATURES_PATH, "w", encoding="utf-8") as f:
        json.dump(
            {
                "numeric_features": NUMERIC_FEATURES,
                "categorical_features": CATEGORICAL_FEATURES,
                "target": TARGET_COL,
            },
            f,
            ensure_ascii=False,
            indent=2,
        )
    print(f"✅ تم حفظ تعريف الأعمدة في: {FEATURES_PATH}")

### 6. Main Function

In [ ]:
def main():
    print("1) قراءة البيانات...")
    df = load_data(DATA_PATH)

    print("2) تجهيز البيانات...")
    df_proc = preprocess(df)
    print(f"عدد السجلات بعد التنظيف: {len(df_proc)}")

    print("3) تدريب وتقييم المودل...")
    pipeline = train_and_evaluate(df_proc)

    print("4) حفظ المخرجات...")
    save_artifacts(pipeline)

    print("\n🎉 جاهز: تم تدريب وحفظ مودل YAQITH Sentinel بنجاح.")


if __name__ == "__main__":
    main()

1) قراءة البيانات...
2) تجهيز البيانات...
عدد السجلات بعد التنظيف: 2300
3) تدريب وتقييم المودل...
🚀 Training model...

=== Confusion Matrix ===
[[343  29]
 [137  66]]

=== Classification Report ===
              precision    recall  f1-score   support

           0      0.715     0.922     0.805       372
           1      0.695     0.325     0.443       203

    accuracy                          0.711       575
   macro avg      0.705     0.624     0.624       575
weighted avg      0.708     0.711     0.677       575

4) حفظ المخرجات...

✅ تم حفظ الـ pipeline في: /content/models/risk_model_pipeline.joblib
✅ تم حفظ تعريف الأعمدة في: /content/models/yaqith_features.json

🎉 جاهز: تم تدريب وحفظ مودل YAQITH Sentinel بنجاح.


### 1. Importing Packages & Identification

In [ ]:
# Importing all necessary packages for AI Agent
from __future__ import annotations

from dataclasses import dataclass
from typing import Literal, List, Dict, Any

import joblib
import pandas as pd

In [ ]:
ActionType = Literal["allow", "step_up", "block"]
RiskLevelType = Literal["low", "medium", "high"]

### 2. Login Event Identification

In [ ]:
@dataclass
class LoginEvent:
    event_id: int
    user_id: int
    device_type: str
    device_id: str
    city: str
    country: str
    is_vpn: int
    is_new_device: int
    is_new_city: int
    distance_from_last_login_km: float
    hour_of_day: int
    weekday: int
    failed_logins_last_24h: int
    typing_speed_cps: float
    nav_pattern_similarity: float
    session_duration_sec: float
    past_compromised_account: int

    @classmethod
    def from_row(cls, row: pd.Series) -> "LoginEvent":
        """إنشاء LoginEvent من سطر DataFrame."""
        return cls(
            event_id=int(row["event_id"]),
            user_id=int(row["user_id"]),
            device_type=str(row["device_type"]),
            device_id=str(row["device_id"]),
            city=str(row["city"]),
            country=str(row["country"]),
            is_vpn=int(row["is_vpn"]),
            is_new_device=int(row["is_new_device"]),
            is_new_city=int(row["is_new_city"]),
            distance_from_last_login_km=float(row["distance_from_last_login_km"]),
            hour_of_day=int(row["hour_of_day"]),
            weekday=int(row["weekday"]),
            failed_logins_last_24h=int(row["failed_logins_last_24h"]),
            typing_speed_cps=float(row["typing_speed_cps"]),
            nav_pattern_similarity=float(row["nav_pattern_similarity"]),
            session_duration_sec=float(row["session_duration_sec"]),
            past_compromised_account=int(row["past_compromised_account"]),
        )

### 3. Security Decision Identification

In [ ]:
# This will be the output of the AI Agent after evaluating the Login Event
@dataclass
class SecurityDecision:
    action: ActionType            # "allow" / "step_up" / "block"
    risk_score: float             # Value between 0 and 1
    risk_level: RiskLevelType     # "low" / "medium" / "high"
    reasons: List[str]            #  Reasons behind the decision
    metadata: Dict[str, Any]      #  (user_id, device_id, city, ...)

### 4. YAQITH Sentinel Agent Identification

In [22]:
NUMERIC_FEATURES + CATEGORICAL_FEATURES

['is_vpn',
 'is_new_device',
 'is_new_city',
 'distance_from_last_login_km',
 'hour_of_day',
 'weekday',
 'failed_logins_last_24h',
 'typing_speed_cps',
 'nav_pattern_similarity',
 'session_duration_sec',
 'past_compromised_account',
 'device_type',
 'city',
 'country']

In [23]:
class YAQITHSentinelAgent:
    """
    الوكيل الأمني الرئيسي (AI Agent):
    - يستقبل LoginEvent
    - يبني Features بنفس شكل التدريب
    - يستخدم الـ pipeline لحساب احتمالية الهجوم
    - يقرر الإجراء المناسب (allow / step_up / block)
    """

    def __init__(
        self,
        model_path: str = "/content/models/risk_model_pipeline.joblib",
        low_to_medium_threshold: float = 0.3,
        medium_to_high_threshold: float = 0.7,
    ) -> None:
        self.model_path = model_path
        self.low_to_medium_threshold = low_to_medium_threshold
        self.medium_to_high_threshold = medium_to_high_threshold

        try:
            self.pipeline = joblib.load(model_path)
            print(f"✅ تم تحميل الـ pipeline من: {model_path}")
        except FileNotFoundError:
            self.pipeline = None
            print(
                f"[WARNING] لم يتم العثور على {model_path}. "
                f"سيتم استخدام قواعد بسيطة لحساب المخاطر (rule-based)."
            )

    # --------- بناء Features للـ pipeline ---------

    def _build_feature_frame(self, event: LoginEvent) -> pd.DataFrame:
        """
        يحول LoginEvent إلى DataFrame بصف واحدة.
        يحتوي على نفس الأعمدة:
        NUMERIC_FEATURES + CATEGORICAL_FEATURES
        """

        data = {
            # numeric
            "is_vpn": [event.is_vpn],
            "is_new_device": [event.is_new_device],
            "is_new_city": [event.is_new_city],
            "distance_from_last_login_km": [event.distance_from_last_login_km],
            "hour_of_day": [event.hour_of_day],
            "weekday": [event.weekday],
            "failed_logins_last_24h": [event.failed_logins_last_24h],
            "typing_speed_cps": [event.typing_speed_cps],
            "nav_pattern_similarity": [event.nav_pattern_similarity],
            "session_duration_sec": [event.session_duration_sec],
            "past_compromised_account": [event.past_compromised_account],

            # categorical
            "device_type": [event.device_type],
            "city": [event.city],
            "country": [event.country],
        }

        return pd.DataFrame(data)

    # --------- حساب درجة الخطر ---------

    def _predict_risk_ml(self, features: pd.DataFrame) -> float:
        """يستخدم الـ pipeline المدرب لحساب احتمالية أن تكون المحاولة هجوم."""
        if self.pipeline is None:
            raise RuntimeError("لا يوجد نموذج ML محمّل (pipeline is None).")

        proba = self.pipeline.predict_proba(features)[0, 1]
        return float(proba)

    def _predict_risk_rule_based(self, event: LoginEvent) -> float:
        """
        بديل بسيط لو ما فيه مودل:
        يحسب risk_score يدويًا من مجموعة إشارات.
        """
        score = 0.0

        if event.is_new_device:
            score += 0.25
        if event.is_new_city:
            score += 0.2
        if event.is_vpn:
            score += 0.2
        if event.past_compromised_account:
            score += 0.3

        if event.failed_logins_last_24h >= 3:
            score += 0.2
        elif event.failed_logins_last_24h == 2:
            score += 0.1

        if event.nav_pattern_similarity < 0.5:
            score += 0.15
        elif event.nav_pattern_similarity < 0.7:
            score += 0.1

        if event.session_duration_sec < 10:
            score += 0.1

        return float(max(0.0, min(1.0, score)))

    def _get_risk_level(self, risk_score: float) -> RiskLevelType:
        """تحويل risk_score إلى مستوى خطر نصي."""
        if risk_score < self.low_to_medium_threshold:
            return "low"
        elif risk_score < self.medium_to_high_threshold:
            return "medium"
        return "high"

    def _decide_action(self, risk_level: RiskLevelType) -> ActionType:
        """تحديد الإجراء الأمني بناءً على مستوى الخطر."""
        if risk_level == "low":
            return "allow"
        elif risk_level == "medium":
            return "step_up"  # تحقق إضافي (OTP / FaceID)
        return "block"       # high risk

    def _build_reasons(
        self,
        event: LoginEvent,
        risk_score: float,
        risk_level: RiskLevelType,
    ) -> List[str]:
        """تجميع أسباب القرار (Explainability)."""
        reasons: List[str] = []

        if event.is_new_device:
            reasons.append("محاولة تسجيل دخول من جهاز جديد.")
        if event.is_new_city:
            reasons.append("محاولة تسجيل دخول من مدينة جديدة.")
        if event.is_vpn:
            reasons.append("VPN استخدام")
        if event.failed_logins_last_24h > 0:
            reasons.append(f"{event.failed_logins_last_24h} محاولات فاشلة خلال آخر 24 ساعة.")
        if event.past_compromised_account:
            reasons.append("الحساب سبق وتم تصنيفه كمعرّض للاختراق.")
        if event.nav_pattern_similarity < 0.7:
            reasons.append("نمط التنقل مختلف عن سلوك المستخدم المعتاد.")
        if event.session_duration_sec < 10:
            reasons.append("جلسة سريعة جدًا، قد تدل على نشاط آلي أو اختبار وصول فقط.")

        if not reasons:
            reasons.append("المخاطر منخفضة ولا توجد إشارات سلوكية مقلقة.")

        reasons.append(f"درجة الخطر المحسوبة: {risk_score:.2f} (مستوى: {risk_level}).")
        return reasons

    # --------- الدالة الرئيسية ---------

    def assess_login(self, event: LoginEvent) -> SecurityDecision:
        """
        الدالة الأساسية:
        - تبني features
        - تحسب risk
        - تحدد level + action
        - ترجع قرار أمني كامل.
        """
        features = self._build_feature_frame(event)

        if self.pipeline is not None:
            risk_score = self._predict_risk_ml(features)
        else:
            risk_score = self._predict_risk_rule_based(event)

        risk_level = self._get_risk_level(risk_score)
        action = self._decide_action(risk_level)
        reasons = self._build_reasons(event, risk_score, risk_level)

        metadata: Dict[str, Any] = {
            "event_id": event.event_id,
            "user_id": event.user_id,
            "device_id": event.device_id,
            "device_type": event.device_type,
            "city": event.city,
            "country": event.country,
        }

        return SecurityDecision(
            action=action,
            risk_score=risk_score,
            risk_level=risk_level,
            reasons=reasons,
            metadata=metadata,
        )

## Demo to try the YAQITH Sentinel Agent

In [34]:
# ديمو عملي: نجرب الوكيل على سجل عشوائي من البيانات

DATA_PATH = "/content/yaqith_sentinel_synthetic_logins_All.xlsx"

df_demo = pd.read_excel(DATA_PATH)

# نختار محاولة تسجيل دخول عشوائية
sample_row = df_demo.sample(1).iloc[0]

event = LoginEvent.from_row(sample_row)

agent = YAQITHSentinelAgent(
    model_path="/content/models/risk_model_pipeline.joblib",
    low_to_medium_threshold=0.3,
    medium_to_high_threshold=0.7,
)

decision = agent.assess_login(event)

print("=== Security Decision ===")
print(f"Action     : {decision.action}")
print(f"Risk score : {decision.risk_score:.3f}")
print(f"Risk level : {decision.risk_level}")

print("\nReasons:")
for r in decision.reasons:
    print(f"- {r}")

print("\nMetadata:")
for k, v in decision.metadata.items():
    print(f"- {k}: {v}")

✅ تم تحميل الـ pipeline من: /content/models/risk_model_pipeline.joblib
=== Security Decision ===
Action     : allow
Risk score : 0.080
Risk level : low

Reasons:
- المخاطر منخفضة ولا توجد إشارات سلوكية مقلقة.
- درجة الخطر المحسوبة: 0.08 (مستوى: low).

Metadata:
- event_id: 1321
- user_id: 279
- device_id: I163
- device_type: ios
- city: Riyadh
- country: Saudi Arabia
